In [1]:
import torch
from torchvision.utils import make_grid
import torch.nn.functional as F
import matplotlib.pyplot as plt

from ultralytics import RTDETR
from ultralytics.utils import SETTINGS
import torch

import numpy as np
np.set_printoptions(suppress=True)

print(SETTINGS)

JSONDict("/home/lucas/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/lucas/capstone/dataset_detection",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "646906117929847a97c427b7cd47692d883d69e32d8292a4b142839e32aa7113",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


In [2]:
def resize_image(image_tensor, size):
    return F.interpolate( image_tensor.unsqueeze(0), size = ( size, size ),
                          mode = 'bilinear', align_corners = False ).squeeze(0)

def pad_image(image_tensor, target_size=64):
    """
    Pads the input image tensor with a black border so that the final image
    has dimensions (target_size, target_size). The image is centered.
    
    Assumes image_tensor is of shape [C, H, W].
    
    Parameters:
    - image_tensor: torch.Tensor of shape [C, H, W]
    - target_size: Desired output size (both height and width)
    
    Returns:
    - padded_image: torch.Tensor of shape [C, target_size, target_size]
    """
    # Get original image dimensions
    _, H, W = image_tensor.shape
    
    # Compute the padding sizes for height and width
    pad_height = target_size - H
    pad_width = target_size - W
    
    # Ensure that the image is smaller than the target size
    if pad_height < 0 or pad_width < 0:
        raise ValueError("The image dimensions are larger than the target size.")
    
    # Calculate padding for each side (left, right, top, bottom)
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    
    # F.pad expects the padding tuple in the order: (pad_left, pad_right, pad_top, pad_bottom)
    padded_image = F.pad(image_tensor, (pad_left, pad_right, pad_top, pad_bottom), mode='constant', value=0)
    return padded_image

In [3]:
# model = RTDETR('yolo/rtdetr-x.pt' )
model = RTDETR('yolo/runs/train/rtdetr_custom_4/weights/best.pt' )

In [4]:
# feature_maps = None
# def hook_fn(module, input, output):
#     global feature_maps
#     # feature_maps = input[0]
#     feature_maps = output.detach().cpu()

In [5]:
# model.model.model[1].register_forward_hook(hook_fn)
# r = model( 'front_1706545937.png' )
# layer_index = 0
# t_ = feature_maps[0].cpu().permute( 1, 2, 0 )
# plt.figure( figsize = ( 8, 8 ) )
# plt.imshow( t_[:,:,layer_index], cmap = 'magma' )
# plt.show()
# t_.min(), t_.max()

In [6]:
torch.save( model.state_dict, 'models/pre_trained/rtdetr-c.pth' )

In [ ]:
# model( 'dataset_detection/images/train/front_1706545937.png' )

In [ ]:
# model.forward( torch.zeros((1, 3, 640, 640)) )  # dry run

In [ ]:
model.train(
    data = 'dataset_detection/data.yaml',  # Path to your dataset configuration file
    # data = 'dataset_detection/data.yaml',  # Path to your dataset configuration file
    epochs = 20,
    imgsz = 640,
    batch = 16,
    project = 'yolo/runs/train',
    name = 'rtdetr_custom_4',
    exist_ok = True,
    copy_paste = 0.5,
    degrees = 10.0,       # Random rotation degrees
    translate = 0.1,      # Random translation fraction
    scale = 0.5,          # Random scale factor
    # shear = 2.0,          # Random shear degrees
    flipud = 0.5,         # Horizontal flip probability
    fliplr = 0.5,         # Vertical flip probability
    hsv_h = 0.015,        # HSV hue augmentation
    hsv_s = 0.7,          # HSV saturation augmentation
    hsv_v = 0.4,          # HSV value augmentation
    mosaic = 0.2,         # Mosaic augmentation probability
    mixup = 0.2           # MixUp augmentation probability
)